<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/%E5%AF%B9%E8%AF%9D%E5%92%8C%E6%91%98%E8%A6%81%E9%87%8D%E7%BB%84%E5%B0%8F%E8%AF%B4_%E4%B8%A4%E7%A7%8D%E6%96%B9%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 由对话和摘要重组小说
## created by 李鲁鲁 and 米唯实

- [Done] 尝试定位summrization在原文本中的位置
- [Done] 尝试定位dialogue在原文本中的位置
- [Done] 将定位后的文本重建为jsonl文件
- [Done] 导入切分好的chunk

## 先载入库和文本，以笑傲江湖为例

In [ ]:
!pip -q install langchain openai google-search-results tiktoken
!pip -q install kor markdownify


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.3 MB/s eta 0:00:00


## 导入原小说，并放在/content下

In [ ]:
# !wget https://gist.githubusercontent.com/zhenghaoz/7ad0948d5835468844abd2db8f67faff/raw/98bd7a3ca7ef308b031a20a0c671b2d2da23ea3e/%25E5%25A4%25A9%25E9%25BE%2599%25E5%2585%25AB%25E9%2583%25A8.txt

!wget https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/%E7%AC%91%E5%82%B2%E6%B1%9F%E6%B9%96.txt

--2023-09-01 09:56:22--  https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/%E7%AC%91%E5%82%B2%E6%B1%9F%E6%B9%96.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2969642 (2.8M) [text/plain]
Saving to: ‘笑傲江湖.txt’

笑傲江湖.txt        100%[===================>]   2.83M  --.-KB/s    in 0.08s   

2023-09-01 09:56:23 (34.4 MB/s) - ‘笑傲江湖.txt’ saved [2969642/2969642]



## 章节切割
***请修改为你的小说的名字***

如果你想导入切分好的chunk，请运行下方导入chunk_list

In [ ]:
import re

raw_text = open('/content/笑傲江湖.txt', encoding='utf-8').read()

chapter_pattern = r'[^\n]+第\d+章[^\n]+\n'

chapters = re.split(chapter_pattern, raw_text)

print(len(chapters))

# chapters = []
# for match in re.finditer(r'\n第.+?章(.+?)\n', raw_text):
#     title = match.group(1)
#     start = match.start()
#     end = raw_text.find('\n第', start+1)
#     chapters.append(raw_text[start:end])

41


In [ ]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

## 递推切割成接近于1500个token的chunk

In [ ]:
def divide_str(s, sep=['\n', '.', '。']):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, 0, mid_len)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos -
                                                        mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]


def strong_divide(s):
    left, right = divide_str(s)

    if right != '':
        return left, right

    whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']
    left, right = divide_str(s, sep=whole_sep)

    if right != '':
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]


In [ ]:
max_token_len = 1500
chunk_text = []


flag = True

for chapter in chapters:

    split_text = chapter.split('\n')

    curr_len = 0
    curr_chunk = ''

    tmp = []

    for line in split_text:
        line_len = len(enc.encode( line ))

        if line_len <= max_token_len - 5:
            tmp.append(line)
        else:
            # print('divide line with length = ', line_len)
            path = [line]
            tmp_res = []

            while path:
                my_str = path.pop()
                left, right = strong_divide(my_str)

                len_left = len(enc.encode( left ))
                len_right = len(enc.encode( right ))

                if len_left > max_token_len - 15:
                    path.append(left)
                else:
                    tmp_res.append(left)

                if len_right > max_token_len - 15:
                    path.append(right)
                else:
                    tmp_res.append(right)

            for line in tmp_res:
                tmp.append(line)

    split_text = tmp

    for line in split_text:
        line_len = len(enc.encode( line ))

        if line_len > max_token_len:
            print('warning line_len = ', line_len)

        if curr_len + line_len <= max_token_len:
            curr_chunk += line
            curr_chunk += '\n'
            curr_len += line_len
            curr_len += 1
        else:
            chunk_text.append(curr_chunk)
            curr_chunk = line
            curr_len = line_len

    if curr_chunk:
        chunk_text.append(curr_chunk)

    # break

print(len(chunk_text))

976


## 如果你想通过切分好的chunk

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = f"/content/drive/MyDrive/shediaoyingxiongzhuan_extract"
# chunk_text = []

import os

chunk_text = [""] * (sum([1 for file_name in os.listdir(folder_path) if file_name.endswith("_sum.txt")]))

# 遍历文件夹中的文件
for file_name in os.listdir(folder_path):
    if file_name.endswith("_raw.txt"):
        file_path = os.path.join(folder_path, file_name)

        # 提取文件名中的 i
        i = int(file_name.split('_')[0])

        # 打开文件并读取内容
        with open(file_path, 'r') as file:
            text = file.read()

        # 将文件内容添加到列表中，按 i 的顺序插入到正确位置
        chunk_text[i] = text

print(len(chunk_text))

929


## 导入抽取出来的dialogue和sum文件，成为一个list

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

保存的路径可以修改

In [ ]:
!rm -rf /content/drive/MyDrive/reorganized_story_shediaoyingxiongzhuan
!mkdir /content/drive/MyDrive/reorganized_story_shediaoyingxiongzhuan

In [ ]:
import numpy as np
import os
import copy
import json

## 手动导入ID，继续向下运行以测试，最后根据ID循环

In [ ]:
id = 200

story_name_en = 'shediaoyingxiongzhuan'
raw_text = chunk_text[ id ]

import os

save_folder = f"/content/drive/MyDrive/{story_name_en}_extract"

dialoge_file = os.path.join(save_folder, f"{id}_dialogue.txt")
summarzie_file = os.path.join(save_folder, f"{id}_sum.txt")

chunk_sum = []
unique_chunk_sum = []
# 给定summarzie_file = os.path.join(save_folder, f"{id}_sum.txt")

# 先检查这个文件是否存在

# 然后使用utf-8编码打开，检查每一行，如果strip后，行首是'-'，则把后面的字符串append到一个list chunk_sum中

# 请用python为我实现
if os.path.exists(summarzie_file):
    with open(summarzie_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip().startswith('-'):
                chunk_sum.append(line.strip()[1:].strip())
if os.path.exists(dialoge_file):
  with open(dialoge_file, encoding='utf-8') as f:
    dialogues = []
    for line in f:
        dialogue = json.loads(line)
        dialogues.append(dialogue)

unique_dialogue = []
for item in dialogues:
    if item not in unique_dialogue:
        unique_dialogue.append(item)
dia_texts = [data['dialogue'] for data in unique_dialogue]
for item in chunk_sum:
    if item not in unique_chunk_sum:
        unique_chunk_sum.append(item)

chunk_sum = unique_chunk_sum
dialogues = unique_dialogue
print(dialogues)
print(dia_texts)
print(chunk_sum)

[{'role': '郭靖', 'dialogue': '郭靖一踏进房，便觉药气冲鼻，又见桌上、榻上、地下，到处放满了诸般药材，以及大大小小的瓶儿、罐儿、缸儿、钵儿，看来梁子翁喜爱调弄丹药，虽在客中，也不放下这些家伙。那小童显也熟习药性，取了四味药，用白纸分别包了，交给简管家。'}, {'role': '郭靖', 'dialogue': '郭靖伸手接过，转身出房。他药已到手，不再看住简管家。不料这管家甚是狡猾，出房时故意落后，待郭靖与那小童一出门，立时将门关上，撑上门闩，大声叫喊：“有贼啊，有贼啊”郭靖一怔，转身推门，那门甚是坚实，一时推之不开。那青衣童子年纪虽小，却机怜异常，听得简管家叫喊，知道不妙，乘郭靖使力推门之际，夹手抢过他手中那四包药，往旁边池塘中一丢。郭靖击出两掌，居然都给他闪避开去。'}, {'role': '郭靖', 'dialogue': '郭靖又惊又怒，双掌按在门上，运起内力，喀喇一响，门闩立时崩断。'}, {'role': '郭靖', 'dialogue': '他抢进门去，一拳击在简管家下颚之上，颚骨登时碎裂，哪里还能做声幸好梁子翁性喜僻静，居处指定要与别的房舍远离，那简管家这几下叫唤，倒无旁人听到。'}, {'role': '郭靖', 'dialogue': '他回身出门，见那童子已奔在数丈之外，急忙提气纵身，霎时间已追到身后，伸手往他后领抓落。那童子听得脑后风响，身子一挫，右腿横扫，身手竟自不弱。郭靖知道只要给他声张出来，不但药物不能得手，而且黄蓉与自己尚有性命之忧，下手更不容情，钩、拿、抓、打，招招是分筋错骨手的狠辣家数。'}, {'role': '郭靖', 'dialogue': '那童子跟着梁子翁，到处受人尊敬，从未遇过强敌，这时不觉心慌意乱，脸上连中了两拳。郭靖乘势直上，拍的一记，又在他天灵盖上击了一掌，那童子立时昏晕过去。郭靖提足将他拨入路旁草丛，回进房去，打火点亮蜡烛，见那简管家倒在地下，兀自昏晕。'}, {'role': '郭靖', 'dialogue': '郭靖暗骂自己胡涂：“那童儿刚才从哪四个瓶罐里取药，我可全没留意，现今怎知这四味药放在哪里”但见瓶罐上面画的都是些弯弯曲曲的符号，竟无一个文字，心下好生为难：“记得他是站在这里拿的，我且把这个角落里的数十罐药每样都拿些，回头请王道长选出来就是。”取过一叠白纸，每样药材都包了

In [ ]:
# 给定长文本raw_text。

# 使用换行符\n或者。 来对这个字符串进行切割，忽略掉strip之后是空的子字符串

# 将每一段话的起点位置存储在一个list of int , starts中

# 将每一段话的结束位置存储在一个list of int , ends中

# 并且将每一个子字符串的存储在一个list of str, lines中

def divide_raw2lines(raw_text):
  previous_str = ''
  starts = []
  ends = []
  lines = []
  for i in range(len(raw_text)):
      previous_str += raw_text[i]
      if raw_text[i] in ('\n','。'):
          strip_str = previous_str.strip(' "“”\r\n')
          if len(strip_str)>0:
              lines.append(strip_str)
              starts.append(i - len(strip_str))
              ends.append(i)
          previous_str = ''
      else:
          pass
  return lines , starts , ends


lines, starts, ends = divide_raw2lines(raw_text)

print(len(lines))


20


In [ ]:
print(lines)

['郭靖一踏进房，便觉药气冲鼻，又见桌上、榻上、地下，到处放满了诸般药材，以及大大小小的瓶儿、罐儿、缸儿、钵儿，看来梁子翁喜爱调弄丹药，虽在客中，也不放下这些家伙。', '那小童显也熟习药性，取了四味药，用白纸分别包了，交给简管家。', '郭靖伸手接过，转身出房。', '他药已到手，不再看住简管家。', '不料这管家甚是狡猾，出房时故意落后，待郭靖与那小童一出门，立时将门关上，撑上门闩，大声叫喊：“有贼啊，有贼啊”郭靖一怔，转身推门，那门甚是坚实，一时推之不开。', '那青衣童子年纪虽小，却机怜异常，听得简管家叫喊，知道不妙，乘郭靖使力推门之际，夹手抢过他手中那四包药，往旁边池塘中一丢。', '郭靖击出两掌，居然都给他闪避开去。', '郭靖又惊又怒，双掌按在门上，运起内力，喀喇一响，门闩立时崩断。', '他抢进门去，一拳击在简管家下颚之上，颚骨登时碎裂，哪里还能做声幸好梁子翁性喜僻静，居处指定要与别的房舍远离，那简管家这几下叫唤，倒无旁人听到。', '他回身出门，见那童子已奔在数丈之外，急忙提气纵身，霎时间已追到身后，伸手往他后领抓落。', '那童子听得脑后风响，身子一挫，右腿横扫，身手竟自不弱。', '郭靖知道只要给他声张出来，不但药物不能得手，而且黄蓉与自己尚有性命之忧，下手更不容情，钩、拿、抓、打，招招是分筋错骨手的狠辣家数。', '那童子跟着梁子翁，到处受人尊敬，从未遇过强敌，这时不觉心慌意乱，脸上连中了两拳。', '郭靖乘势直上，拍的一记，又在他天灵盖上击了一掌，那童子立时昏晕过去。', '郭靖提足将他拨入路旁草丛，回进房去，打火点亮蜡烛，见那简管家倒在地下，兀自昏晕。', '郭靖暗骂自己胡涂：“那童儿刚才从哪四个瓶罐里取药，我可全没留意，现今怎知这四味药放在哪里”但见瓶罐上面画的都是些弯弯曲曲的符号，竟无一个文字，心下好生为难：“记得他是站在这里拿的，我且把这个角落里的数十罐药每样都拿些，回头请王道长选出来就是。', '取过一叠白纸，每样药材都包了一包，生怕刚才简管家叫喊时被人听见，心里一急，包得更加慢了。', '好容易在每个药瓶中都取了药包好，揣在怀里，大功告成，心下欢喜，回过身来，不提防手肘在旁边的大竹篓上一撞。', '那竹篓横跌翻倒，盖子落下，蓦地呼噜一声，窜出一条殷红如血的大蛇，猛向他脸上扑来。', '郭靖大吃一惊，急忙向后纵开，只见那蛇身子有小

In [ ]:
# 已知if '\u4e00' <= char <= '\u9fa5': 可以判断一个char是否是中文字

# 我希望实现一个函数，这个函数的输入是两个list of string, 长度为M的query 和 长度为N的datas

# 输出是一个M*N的numpy float数组 recalls

# 先计算freqs[m][n] 表示query的第m句中的每一个中文字，是否在datas[n]中是否出现，如果出现，则freqs[m][n]加一

# 然后计算recalls[m][n]是freqs[m][n]除掉 query[m]中所有中文字的个数

def compute_char_recall(query, datas):
    M = len(query)
    N = len(datas)

    freqs = np.zeros((M, N), dtype=int)

    for m in range(M):
        q_chars = set()
        for char in query[m]:
            if '\u4e00' <= char <= '\u9fa5':
                q_chars.add(char)

        for n in range(N):
            for char in q_chars:
                if char in datas[n]:
                    freqs[m][n] += 1

    query_chars_count = [len(set(char for char in sent if '\u4e00'<= char <= '\u9fa5'))
                         for sent in query]

    recalls = freqs / np.array(query_chars_count)[:, None]

    return recalls

## 可视化用代码，匹配时可以不运行

In [ ]:
import plotly.express as px

s = compute_char_recall(chunk_sum, lines)

fig = px.imshow(s,
                labels=dict(x="Line", y="Summary", color="Recall"),
                x=list(range(len(lines))),
                y=list(range(len(chunk_sum))),
                color_continuous_scale='YlOrRd')

fig.update_xaxes(side="top")

fig.show()

In [ ]:
# import copy
def summary2line(chunk_sum, lines):


  s = compute_char_recall(chunk_sum, lines)

  color_map = {}
  ans_Q = {}

  ans_div = {}

  flags = {}

  M = len( chunk_sum )
  N = len( lines )


  for n in range(0, N):
      if n==0:
          ans_Q[ (0,0) ] = s[0,0]
          ans_div[ (0,0 ) ] = []
      else:
          ans_Q[ (0,n) ] = ans_Q[ (0,n-1) ] + s[0,n]
          ans_div[ (0,n) ] = []

  for m in range(1,M):
      ans_Q[(m,m)] = ans_Q[(m-1,m-1)] + s[m,m]
      ans_div[ (m,m) ] =  ans_div[ (m-1,m-1) ].copy()
      ans_div[ (m,m) ].append(m)


  def find_Q( m , n ):
      # print(m,n)

      if m < 0 or n < 0:
          print('error out bound', m , ' ' , n )
          return 0, []

      if (m,n) in ans_Q.keys():
          return ans_Q[(m,n)], ans_div[(m,n)]

      if (m,n) in color_map.keys():
          print('error repeated quest ', m , ' ', n )
          return 0, []
      else:
          color_map[(m,n)] = 1

      current_div = []

      left, left_div = find_Q( m, n-1 )
      right, right_div = find_Q( m-1, n-1 )

      if left > right:
          ans = left + s[m][n]
          flags[(m,n)] = False
          current_div = left_div

      else:
          ans = right + s[m][n]
          flags[(m,n)] = True
          current_div = right_div.copy()
          current_div.append(n-1)

      # ans = max(  , ) + s[m][n]

      ans_Q[(m,n)] = ans
      ans_div[(m,n)] = current_div.copy()

      return ans, current_div

  # print(find_Q(0,5))
  # print(find_Q(M-1,N-1))

  score, divs = find_Q(M-1,N-1)
  divs.append(N-1)

  return score, divs

score, divs = summary2line(chunk_sum, lines)
print(score, divs)

last = 0

# divs.append(N-1)

for i,div in enumerate(divs):
    print('\n###')
    print(chunk_sum[i])
    print('\n')

    for j in range(last, div):
        print(lines[j])
        last = div

7.196500558265264 [1, 4, 5, 8, 11, 14, 19]

###
郭靖进入房间，发现满满都是药材和药瓶


郭靖一踏进房，便觉药气冲鼻，又见桌上、榻上、地下，到处放满了诸般药材，以及大大小小的瓶儿、罐儿、缸儿、钵儿，看来梁子翁喜爱调弄丹药，虽在客中，也不放下这些家伙。

###
简管家故意将门关上，喊贼引起郭靖的注意


那小童显也熟习药性，取了四味药，用白纸分别包了，交给简管家。
郭靖伸手接过，转身出房。
他药已到手，不再看住简管家。

###
郭靖推门不开，青衣童子夺走了药包并扔进池塘


不料这管家甚是狡猾，出房时故意落后，待郭靖与那小童一出门，立时将门关上，撑上门闩，大声叫喊：“有贼啊，有贼啊”郭靖一怔，转身推门，那门甚是坚实，一时推之不开。

###
郭靖打破门闩，击败简管家


那青衣童子年纪虽小，却机怜异常，听得简管家叫喊，知道不妙，乘郭靖使力推门之际，夹手抢过他手中那四包药，往旁边池塘中一丢。
郭靖击出两掌，居然都给他闪避开去。
郭靖又惊又怒，双掌按在门上，运起内力，喀喇一响，门闩立时崩断。

###
郭靖追赶青衣童子，展开激烈的战斗


他抢进门去，一拳击在简管家下颚之上，颚骨登时碎裂，哪里还能做声幸好梁子翁性喜僻静，居处指定要与别的房舍远离，那简管家这几下叫唤，倒无旁人听到。
他回身出门，见那童子已奔在数丈之外，急忙提气纵身，霎时间已追到身后，伸手往他后领抓落。
那童子听得脑后风响，身子一挫，右腿横扫，身手竟自不弱。

###
郭靖击晕童子，将其藏在草丛中


郭靖知道只要给他声张出来，不但药物不能得手，而且黄蓉与自己尚有性命之忧，下手更不容情，钩、拿、抓、打，招招是分筋错骨手的狠辣家数。
那童子跟着梁子翁，到处受人尊敬，从未遇过强敌，这时不觉心慌意乱，脸上连中了两拳。
郭靖乘势直上，拍的一记，又在他天灵盖上击了一掌，那童子立时昏晕过去。

###
郭靖包好药材，不小心撞倒竹篓，释放出一条红色蛇


郭靖提足将他拨入路旁草丛，回进房去，打火点亮蜡烛，见那简管家倒在地下，兀自昏晕。
郭靖暗骂自己胡涂：“那童儿刚才从哪四个瓶罐里取药，我可全没留意，现今怎知这四味药放在哪里”但见瓶罐上面画的都是些弯弯曲曲的符号，竟无一个文字，心下好生为难：“记得他是站在这里拿的，我且把这个角落里的数十罐药每样都拿些，回头请王道长选出来就是

## 可视化用代码，匹配时可以不运行

In [ ]:
import plotly.express as px

s_dialogue = compute_char_recall(dia_texts, lines)

fig = px.imshow(s_dialogue,
                labels=dict(x="Line", y="Dialogue", color="Recall"),
                x=list(range(len(lines))),
                y=list(range(len(dia_texts))),
                color_continuous_scale='YlOrRd')

fig.update_xaxes(side="top")

fig.show()

s_dialogue 存储了一个M*N的np array

我现在希望实现一个python程序，能够找到一个长度为M的顺序子序列 a_0, ... , a_m-1

使得s_dialogue[ i ][ a_i ] 之和最大

输出a_0, ... , a_{m-1}的值

用动态规划算法，python为我实现


In [ ]:
def dialogue2line(dia_texts, lines):
    s_dialogue = compute_char_recall(dia_texts, lines)

    M, N = s_dialogue.shape
    if M==0 or N==0:
      return []
    dp = np.zeros((M, N))
    dp[0] = s_dialogue[0]
    prev_indices = np.zeros((M, N), dtype=int)
    for i in range(1, M):
        for j in range(N):
            max_prev_index = np.argmax(dp[i-1])
            dp[i][j] = dp[i-1][max_prev_index] + s_dialogue[i][j]
            prev_indices[i][j] = max_prev_index

    max_end_index = np.argmax(dp[-1])
    sequence = []
    for i in range(M-1, -1, -1):
        sequence.append(max_end_index)
        max_end_index = prev_indices[i][max_end_index]
    sequence.reverse()

    return sequence

seq = dialogue2line(dia_texts, lines)
print(seq)

for i,id in enumerate(seq):
    print('\n###')
    print(dia_texts[i])
    print(lines[id])
    print('\n')

    for j in range(last, div):
        print(lines[j])
        last = div

[0, 5, 7, 8, 11, 14, 15, 17, 19]

###
郭靖一踏进房，便觉药气冲鼻，又见桌上、榻上、地下，到处放满了诸般药材，以及大大小小的瓶儿、罐儿、缸儿、钵儿，看来梁子翁喜爱调弄丹药，虽在客中，也不放下这些家伙。那小童显也熟习药性，取了四味药，用白纸分别包了，交给简管家。
郭靖一踏进房，便觉药气冲鼻，又见桌上、榻上、地下，到处放满了诸般药材，以及大大小小的瓶儿、罐儿、缸儿、钵儿，看来梁子翁喜爱调弄丹药，虽在客中，也不放下这些家伙。



###
郭靖伸手接过，转身出房。他药已到手，不再看住简管家。不料这管家甚是狡猾，出房时故意落后，待郭靖与那小童一出门，立时将门关上，撑上门闩，大声叫喊：“有贼啊，有贼啊”郭靖一怔，转身推门，那门甚是坚实，一时推之不开。那青衣童子年纪虽小，却机怜异常，听得简管家叫喊，知道不妙，乘郭靖使力推门之际，夹手抢过他手中那四包药，往旁边池塘中一丢。郭靖击出两掌，居然都给他闪避开去。
那青衣童子年纪虽小，却机怜异常，听得简管家叫喊，知道不妙，乘郭靖使力推门之际，夹手抢过他手中那四包药，往旁边池塘中一丢。



###
郭靖又惊又怒，双掌按在门上，运起内力，喀喇一响，门闩立时崩断。
郭靖又惊又怒，双掌按在门上，运起内力，喀喇一响，门闩立时崩断。



###
他抢进门去，一拳击在简管家下颚之上，颚骨登时碎裂，哪里还能做声幸好梁子翁性喜僻静，居处指定要与别的房舍远离，那简管家这几下叫唤，倒无旁人听到。
他抢进门去，一拳击在简管家下颚之上，颚骨登时碎裂，哪里还能做声幸好梁子翁性喜僻静，居处指定要与别的房舍远离，那简管家这几下叫唤，倒无旁人听到。



###
他回身出门，见那童子已奔在数丈之外，急忙提气纵身，霎时间已追到身后，伸手往他后领抓落。那童子听得脑后风响，身子一挫，右腿横扫，身手竟自不弱。郭靖知道只要给他声张出来，不但药物不能得手，而且黄蓉与自己尚有性命之忧，下手更不容情，钩、拿、抓、打，招招是分筋错骨手的狠辣家数。
郭靖知道只要给他声张出来，不但药物不能得手，而且黄蓉与自己尚有性命之忧，下手更不容情，钩、拿、抓、打，招招是分筋错骨手的狠辣家数。



###
那童子跟着梁子翁，到处受人尊敬，从未遇过强敌，这时不觉心慌意乱，脸上连中了两拳。郭靖乘势直上，拍的一记，又在他天灵盖上击了一掌，那童子立时昏晕过去。郭靖提足将他拨

In [ ]:


print(len(chunk_sum))
print(divs)
print(dia_texts)
print(seq)


# string_indices = [10, 11]
def jsonl_sorted(chunk_sum, divs, dia_texts, seq):

  combined_data = []
  combined_text = ""
  for index in sorted(seq + divs):
      # print(index)
      if index in seq:

          combined_data.append({
              "role" : dialogues[seq.index(index)]["role"],
              'text': dialogues[seq.index(index)]["dialogue"],
              'if_scene': False
          })
          combined_text = combined_text + dialogues[seq.index(index)]["role"] + ":" + dialogues[seq.index(index)]["dialogue"] +"\n"
          seq[seq.index(index)] = -1
      if index in divs:
          combined_data.append({
              "role" : "scene" ,
              'text': chunk_sum[divs.index(index)],
              'if_scene': True
          })
          combined_text = combined_text + "scene" + ":" + chunk_sum[divs.index(index)] +"\n"
          divs[divs.index(index)]=-1

  return combined_data, combined_text

combined_data, combined_text = jsonl_sorted(chunk_sum, divs.copy(), dia_texts, seq.copy())
print(combined_data)
# 打开文件，以写入模式（"w"）打开
with open(f"/content/reorganized_{id}.jsonl", "w", encoding="utf-8") as file:
    # 遍历数据列表中的每个字典
    for record in combined_data:
        # 将字典转换为JSON格式的字符串
        json_record = json.dumps(record, ensure_ascii=False)
        # 将转换后的JSON字符串写入文件，并添加换行符
        file.write(json_record + "\n")


7
[1, 4, 5, 8, 11, 14, 19]
['郭靖一踏进房，便觉药气冲鼻，又见桌上、榻上、地下，到处放满了诸般药材，以及大大小小的瓶儿、罐儿、缸儿、钵儿，看来梁子翁喜爱调弄丹药，虽在客中，也不放下这些家伙。那小童显也熟习药性，取了四味药，用白纸分别包了，交给简管家。', '郭靖伸手接过，转身出房。他药已到手，不再看住简管家。不料这管家甚是狡猾，出房时故意落后，待郭靖与那小童一出门，立时将门关上，撑上门闩，大声叫喊：“有贼啊，有贼啊”郭靖一怔，转身推门，那门甚是坚实，一时推之不开。那青衣童子年纪虽小，却机怜异常，听得简管家叫喊，知道不妙，乘郭靖使力推门之际，夹手抢过他手中那四包药，往旁边池塘中一丢。郭靖击出两掌，居然都给他闪避开去。', '郭靖又惊又怒，双掌按在门上，运起内力，喀喇一响，门闩立时崩断。', '他抢进门去，一拳击在简管家下颚之上，颚骨登时碎裂，哪里还能做声幸好梁子翁性喜僻静，居处指定要与别的房舍远离，那简管家这几下叫唤，倒无旁人听到。', '他回身出门，见那童子已奔在数丈之外，急忙提气纵身，霎时间已追到身后，伸手往他后领抓落。那童子听得脑后风响，身子一挫，右腿横扫，身手竟自不弱。郭靖知道只要给他声张出来，不但药物不能得手，而且黄蓉与自己尚有性命之忧，下手更不容情，钩、拿、抓、打，招招是分筋错骨手的狠辣家数。', '那童子跟着梁子翁，到处受人尊敬，从未遇过强敌，这时不觉心慌意乱，脸上连中了两拳。郭靖乘势直上，拍的一记，又在他天灵盖上击了一掌，那童子立时昏晕过去。郭靖提足将他拨入路旁草丛，回进房去，打火点亮蜡烛，见那简管家倒在地下，兀自昏晕。', '郭靖暗骂自己胡涂：“那童儿刚才从哪四个瓶罐里取药，我可全没留意，现今怎知这四味药放在哪里”但见瓶罐上面画的都是些弯弯曲曲的符号，竟无一个文字，心下好生为难：“记得他是站在这里拿的，我且把这个角落里的数十罐药每样都拿些，回头请王道长选出来就是。”取过一叠白纸，每样药材都包了一包，生怕刚才简管家叫喊时被人听见，心里一急，包得更加慢了。', '好容易在每个药瓶中都取了药包好，揣在怀里，大功告成，心下欢喜，回过身来，不提防手肘在旁边的大竹篓上一撞。那竹篓横跌翻倒，盖子落下，蓦地呼噜一声，窜出一条殷红如血的大蛇，猛向他脸上扑来。', '郭靖大吃一惊，急忙向后纵开，只见那蛇身子有小碗粗细，半身尚在篓中，不知其长几

In [ ]:
with open(f"/content/reorganized_{id}.txt", "w", encoding="utf-8") as file:
    # 遍历数据列表中的每个字典
  file.write(combined_text)

清空储存文件夹

In [ ]:
!rm -rf /content/drive/MyDrive/reorganized_story_yitiantulongji
!mkdir /content/drive/MyDrive/reorganized_story_yitiantulongji

## 以上是手动调整ID版程序，下面是自动化程序，请保证上述单元格除可视化外均已运行，并测试无报错

In [ ]:
import numpy as np
import os
import copy
import json
from tqdm import tqdm

final_jsonl = []
final_txt = ""

for i in tqdm(range(1,len(chunk_text)), desc="Processing", total=len(chunk_text)-1, unit="item"):

  try:
    story_name_en = 'shediaoyingxiongzhuan'
    raw_text = chunk_text[ i ]

    import os

    save_folder = f"/content/drive/MyDrive/{story_name_en}_extract"

    dialoge_file = os.path.join(save_folder, f"{i}_dialogue.txt")
    summarzie_file = os.path.join(save_folder, f"{i}_sum.txt")

    chunk_sum = []
    unique_chunk_sum = []
    if os.path.exists(summarzie_file):
        with open(summarzie_file, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip().startswith('-'):
                    chunk_sum.append(line.strip()[1:].strip())
    if os.path.exists(dialoge_file):
      with open(dialoge_file, encoding='utf-8') as f:
        dialogues = []
        for line in f:
            dialogue = json.loads(line)
            dialogues.append(dialogue)

    unique_dialogue = []
    for item in dialogues:
        if item not in unique_dialogue:
            unique_dialogue.append(item)
    dia_texts = [data['dialogue'] for data in unique_dialogue]
    for item in chunk_sum:
        if item not in unique_chunk_sum:
            unique_chunk_sum.append(item)

    chunk_sum = unique_chunk_sum
    dialogues = unique_dialogue
    lines, starts, ends = divide_raw2lines(raw_text)
    # print(chunk_sum)
    # print(lines)
    score, divs = summary2line(chunk_sum, lines)  #summary匹配
    seq = dialogue2line(dia_texts, lines) #对话匹配
    combined_data, combined_text = jsonl_sorted(chunk_sum, divs.copy(), dia_texts, seq.copy())
    # 如果需要保存每个chunk的，在此处保存
    final_jsonl.append(combined_data)
    final_txt = final_txt + combined_text + "\n"
  except:
    print("第" + str(i) + "个chunk出错")
    pass
with open(f"/content/drive/MyDrive/reorganized_story_{story_name_en}/reorganized_{story_name_en}.jsonl", "w", encoding="utf-8") as file:
  # 遍历数据列表中的每个字典
  for record in final_jsonl:
      # 将字典转换为JSON格式的字符串
      json_record = json.dumps(record, ensure_ascii=False)
      # 将转换后的JSON字符串写入文件，并添加换行符
      file.write(json_record + "\n")
with open(f"/content/drive/MyDrive/reorganized_story_{story_name_en}/reorganized_{story_name_en}.txt", "w", encoding="utf-8") as file:
  file.write(final_txt)

Processing:   1%|▏         | 13/928 [00:00<00:07, 127.40item/s]<ipython-input-27-388c8ce4bd20>:31: RuntimeWarning:

invalid value encountered in divide

Processing:  41%|████      | 381/928 [00:04<00:04, 116.62item/s]

第362个chunk出错


Processing:  94%|█████████▍| 876/928 [00:10<00:00, 91.57item/s]

第861个chunk出错


Processing: 100%|██████████| 928/928 [00:10<00:00, 85.57item/s]


In [ ]:
with open(f"/content/reorganized_story.txt", "w", encoding="utf-8") as file:
    # 遍历数据列表中的每个字典
  file.write(final_txt)